# Meditation competition

**Author**:

 - Borja Sánchez Leirado borja.sanchez.leirado@gmail.com | [@BorjaSanchez [in]](https://www.linkedin.com/in/borjasanlei/) | [@borjasanlei [Github]](https://github.com/borjasanlei)
---

_This code is Open Source, released under a GPL License._

**Victims**:


## Loading Bitbrain's .CSV EEG into a Pandas dataframe

We follow [**session0-importing-bitbrain-data-into-mne**](./session0-importing-bitbrain-data-into-mne.ipynb) to generate `mne.io.raw` object with EEG data.

In [8]:
import mne
import pandas as pd
import numpy as np

%matplotlib inline

In [9]:
import matplotlib 
matplotlib.rcParams['figure.figsize'] = [12, 6]

In [13]:
def Bitbrain_to_raw(path, channels, sampling_rate):
    '''Loading Bitbrain's .CSV EEG into a mne.io.raw object'''
    
    df = pd.read_csv(path)
    df_mne = df.drop(['timestamp', 'sequence', 'battery', 'flags'], axis=1)

    '''Transpose'''
    data_mne = df_mne.to_numpy().transpose()

    '''Scale'''
    data_mne = data_mne / 1000000

    '''Raw info'''
    channels = channels
    info_mne = mne.create_info(ch_names=channels,ch_types='eeg',sfreq = sampling_rate)

    bbt_raw = mne.io.RawArray(data_mne, info_mne)
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    '''Final Raw object'''
    raw_1020 = bbt_raw.copy().set_montage(ten_twenty_montage)
    
    return raw_1020

In [14]:
'''Test'''
path = "data/02-08-2020-jv-baseline-activity-no-labels-BBT-E16-AAA005-2020-02-08_12-59-21/EEG.csv"
channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C1', 'C3', 'C2', 'C4', 'CP1', 'CP3', 'CP2', 'CP4', 'Cz', 'O1', 'O2', 'Pz']
sampling_rate = 256

In [15]:
rawBitbrain_to_raw(path, channels, sampling_rate)

Creating RawArray with float64 data, n_channels=16, n_times=60712
    Range : 0 ... 60711 =      0.000 ...   237.152 secs
Ready.
DigMontage is a superset of info. 78 in DigMontage will be ignored. The ignored channels are: {'F10', 'C5', 'T4', 'PO4', 'M2', 'F9', 'AF5', 'T6', 'AFz', 'FT8', 'PO8', 'AF1', 'P10', 'P5', 'FC2', 'T10', 'T5', 'FC4', 'F1', 'PO5', 'AF3', 'P4', 'P9', 'TP9', 'Iz', 'P2', 'PO2', 'M1', 'AF4', 'FT7', 'P7', 'Oz', 'AF6', 'P3', 'A2', 'O10', 'POz', 'T8', 'PO1', 'PO3', 'O9', 'FT9', 'FC3', 'A1', 'C6', 'TP8', 'F7', 'F5', 'F6', 'AF9', 'PO10', 'AF8', 'Fz', 'P6', 'P8', 'CP5', 'AF2', 'PO6', 'F2', 'PO9', 'P1', 'AF7', 'FC6', 'TP10', 'T3', 'AF10', 'F8', 'TP7', 'Fpz', 'FC1', 'T7', 'PO7', 'FCz', 'FC5', 'CPz', 'T9', 'CP6', 'FT10'}


<RawArray  |  None, n_channels x n_times : 16 x 60712 (237.2 sec), ~7.5 MB, data loaded>